## Installing libraries

In [1]:
#!pip install node2vec
#!pip install neo4j
#!pip install --upgrade networkx
#!pip install tensorflow

## Importing Libraries

In [11]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances
from neo4j import GraphDatabase
from node2vec import Node2Vec
import networkx as nx
import numpy as np
import scipy.sparse as sp
import matplotlib.pyplot as plt
import gensim.downloader as api
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn.functional as F
#from tensorflow.keras.layers import BatchNormalization, Dropout, ReLU

## Connecting to the neo4j database

In [12]:
uri = "bolt://localhost:7687"
username = "neo4j"
password = "OLIV00%%"

driver = GraphDatabase.driver(uri, auth=(username, password))

## Loading of the KG

In [13]:
graph = nx.Graph()

with driver.session() as session:
    # Récupérer les nœuds avec leur nom et type
    nodes_query = "MATCH (n) RETURN ID(n) AS id, n.name AS name, n.type AS type"
    nodes_result = session.run(nodes_query)

    # Parcourir les résultats et ajouter les nœuds au graphe
    for record in nodes_result:
        node_id = record["id"]
        node_name = record["name"]
        node_type = record["type"]
        if node_id is not None:
            graph.add_node(node_id, name=node_name, type=node_type)

    # Récupérer les relations
    relations_query = "MATCH ()-[r]->() RETURN ID(startNode(r)) AS source_id, ID(endNode(r)) AS target_id"
    relations_result = session.run(relations_query)

    # Ajouter les relations entre les nœuds
    for record in relations_result:
        source_node_id = record["source_id"]
        target_node_id = record["target_id"]
        if source_node_id is not None and target_node_id is not None:
            graph.add_edge(source_node_id, target_node_id)

# Afficher les informations des nœuds
for node_id, node_data in graph.nodes(data=True):
    print("Node ID:", node_id)
    print("Node Name:", node_data.get("name"))
    print("Node Type:", node_data.get("type"))
    print()

Node ID: 0
Node Name: Couple unbalance
Node Type: Explanation

Node ID: 1
Node Name: Dynamic unbalance
Node Type: Explanation

Node ID: 2
Node Name: Overhung unbalance
Node Type: Explanation

Node ID: 3
Node Name: Structural looseness
Node Type: Explanation

Node ID: 4
Node Name: Angular misalignment
Node Type: Explanation

Node ID: 5
Node Name: General misalignment
Node Type: Explanation

Node ID: 6
Node Name: Parallel misalignment
Node Type: Explanation

Node ID: 7
Node Name: Mechanical loosenes
Node Type: Explanation

Node ID: 8
Node Name: Component loosenes
Node Type: Explanation

Node ID: 9
Node Name: Bearing Lubrication Fault
Node Type: Explanation

Node ID: 10
Node Name: Excessive gear
Node Type: Explanation

Node ID: 11
Node Name: Excessive loading
Node Type: Explanation

Node ID: 12
Node Name: Mechanical misalignment
Node Type: Explanation

Node ID: 13
Node Name: Pump
Node Type: Component

Node ID: 14
Node Name: Fan
Node Type: Component

Node ID: 15
Node Name: Engine
Node Type

## Features extraction from nodes

In [14]:
# Préparer les caractéristiques des nœuds
node_features = []

# Récupérer les attributs "id", "name" et "type" des nœuds dans le graphe
for node_id in graph.nodes:
    node_data = graph.nodes[node_id]
    node_feature = {
        "id": node_id,
        "name": node_data["name"],
        "type": node_data["type"]
    }
    node_features.append(node_feature)

# Convertir les caractéristiques des nœuds en un tableau numpy
#X = np.array([(node["id"], node["name"], node["type"]) for node in node_features])
X = np.array(node_features)
print(X[0])

{'id': 0, 'name': 'Couple unbalance', 'type': 'Explanation'}


In [15]:
# Exécutez l'algorithme Node2Vec
node2vec = Node2Vec(graph, dimensions=16, walk_length=15, num_walks=60, workers=4)

# Entraînez le modèle pour apprendre les embeddings
model = node2vec.fit(window=10, min_count=1)

# Obtenez les embeddings pour tous les nœuds du graphe
embeddings = [model.wv[str(node_id)] for node_id in graph.nodes]

Computing transition probabilities:   0%|          | 0/25 [00:00<?, ?it/s]

In [16]:
def transform_array_to_matrix(array):
    """Transforms an array of arrays into a matrix."""
    matrix = np.array(array)
    matrix = matrix.reshape(matrix.shape[0], -1)
    return matrix

X = transform_array_to_matrix(embeddings)
X.shape

(25, 16)

## Attention-based Compressed Relational Graph Convolutional Network (ACRGCN)

In [21]:
def ACRGCN(G, X, L, Decoder_layer):
    '''
    G : Knowledge graph
    X : nodes features (embeddings of the nodes)
    L : Encoder layer number
    Decoder_layer : Decoder layer number
    '''
    H = X
    for i in range(L):   
        H = RGCN(H, G)
        H = BN(H)
        H = ReLU(H)
        if i < L-1:
            H = Dropout(H)
    H_maxlayer = GAT(H, G)
    
    for i in range(Decoder_layer, 0, -1):
        H = DeRGCN(H_maxlayer, G)
        H = BN(H)
        H = ReLU(H)
        if i > 1:
            H = Dropout(H)
        H = np.concatenate((H, H_maxlayer), axis=1)
    
    E_candidate = ScoreDistMult(H)
    
    return E_candidate


def RGCN(H, G):
    A = nx.adjacency_matrix(G)
    for layer in range(L):
        W_layer = np.random.normal(0, 0.01, (H.shape[1], H.shape[1]))
        H = H@W_layer
        H = A.dot(H)
        H = np.maximum(H, 0)
    return H


def BN(H):
    # Batch Normalization operation
    scaler = StandardScaler()
    H = scaler.fit_transform(H)
    return H


def ReLU(H):
    # ReLU activation function
    return np.maximum(H, 0)


def Dropout(H, p=0.5):
    # Dropout operation
    mask = np.random.binomial(1, p, size=H.shape)
    H = H * mask / p
    return H


def GAT(H, G, lambda_val=0.2):
    num_nodes, feature_dim = H.shape
    H = torch.tensor(H, dtype=torch.float32)
    
    # Initialize attention weight matrix (W_a) with random values
    W_a = torch.randn(feature_dim, feature_dim, dtype=torch.float32)
    
    # Convert lambda_val to a PyTorch tensor with an extra dimension
    lambda_val = torch.tensor([lambda_val], dtype=torch.float32)
    H_a = torch.zeros_like(H)
    
    for i in range(num_nodes):
        neighbors = G[i]  # Get the neighbors of the ith node
        num_neighbors = len(neighbors)
        alpha = torch.zeros(num_neighbors)
        
        for j, neighbor_idx in enumerate(neighbors):
            concat_features = torch.cat((torch.matmul(W_a, H[i]), torch.matmul(W_a, H[neighbor_idx])))
            # Ensure lambda_val has the same shape as concat_features for element-wise operations
            expanded_lambda_val = lambda_val.expand_as(concat_features)
            alpha[j] = torch.exp(F.leaky_relu(torch.matmul(expanded_lambda_val, concat_features)))
            
        alpha = F.softmax(alpha, dim=0)
        
        for j, neighbor_idx in enumerate(neighbors):
            H_a[i] += alpha[j] * torch.matmul(W_a, H[neighbor_idx])
            
    H_a = torch.sigmoid(H_a)
    return H_a


def DeRGCN(H, G):
    A = nx.adjacency_matrix(G)
    W_layer = np.random.normal(0, 0.01, (H.shape[1], H.shape[1]))
    H = H@W_layer
    H = A.T.dot(H)
    H = np.maximum(H, 0)
    return H


def ScoreDistMult(H):
    # Score calculation using DistMult
    scores = np.dot(H, np.transpose(H))
    return scores


def Sort(scores):
    # Sort the scores in descending order
    sorted_indices = np.argsort(scores)[::-1] 
    sorted_scores = scores[sorted_indices]
    return sorted_scores

## Model execution

In [22]:
# Appliquer l'algorithme ACRGCN
L = 3  # Encoder layer number
Decoder_layer = 2  # Decoder layer number
E_candidates = ACRGCN(graph, X, L, Decoder_layer)

# Afficher les résultats
print(E_candidates)

IndexError: index 29 is out of bounds for dimension 0 with size 25

In [20]:
E_candidates.shape

NameError: name 'E_candidates' is not defined

## Close session and driver at the end

In [10]:
session.close()
driver.close()

In [ ]:
[0] * 4